# Temporary Impact Analysis and Optimal Execution Algorithm

This notebook aims to:
- Analyze the temporary market impact of trades
- Model the temporary impact function using a power-law
- Formulate and solve the optimal order allocation problem
- Visualize and summarize the results in a professional manner

In [ ]:
# Install matplotlib if not already installed
# %pip install matplotlib

# Import essential libraries for analysis and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import os

# Set up the plots to be clearer and more beautiful
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
# Load real data from zip files for each ticker with error handling


ticker_paths = {

    'CRWV': 'data/CRWV/CRWV.zip',

    'FROG': 'data/FROG/FROG.zip',

    'SOUN': 'data/SOUN/SOUN.zip'

}



ticker_dfs = {}



for ticker, zip_path in ticker_paths.items():

    print(f'\n--- Processing data for {ticker} ---')

    if not os.path.exists(zip_path):

        print(f'❌ File not found: {zip_path}')

        continue

    try:

        with ZipFile(zip_path, 'r') as zip_ref:

            zip_ref.extractall(f'data/{ticker}/extracted')

            csv_files = [f for f in os.listdir(f'data/{ticker}/extracted') if f.endswith(".csv")]

            if not csv_files:

                print(f'❌ No CSV file inside {zip_path}')

                continue

            try:

                df = pd.read_csv(f'data/{ticker}/extracted/' + csv_files[0])

            except Exception as e:

                print(f'❌ Error reading CSV: {e}')

                continue

            required_cols = set(['price', 'volume'])

            if not required_cols.issubset(df.columns):

                print(f'❌ File {csv_files[0]} does not contain required columns: {required_cols - set(df.columns)}')

                continue

            ticker_dfs[ticker] = df

            print(f'✅ Data loaded ({len(df)}) rows')

            display(df.head(5))

    except Exception as e:

        print(f'❌ Error processing {ticker}: {e}')



if not ticker_dfs:

    print('⚠️ No data loaded. Make sure files exist at the correct paths and in the required format.')

In [ ]:
# Data exploration and visual analysis


for ticker, df in ticker_dfs.items():

    print(f'\n--- Analysis for {ticker} ---')

    display(df.describe().T)

    if 'price' in df.columns and 'volume' in df.columns:

        fig, axes = plt.subplots(1, 2, figsize=(14, 4))

        sns.lineplot(ax=axes[0], data=df['price'], color='royalblue')

        axes[0].set_title(f'{ticker} Price')

        axes[0].set_xlabel('Time')

        axes[0].set_ylabel('Price')

        sns.lineplot(ax=axes[1], data=df['volume'], color='darkorange')

        axes[1].set_title(f'{ticker} Trading Volume')

        axes[1].set_xlabel('Time')

        axes[1].set_ylabel('Volume')

        plt.suptitle(f'Visual Analysis for {ticker}', fontsize=14)

        plt.tight_layout()

        plt.show()

NameError: name 'ticker_dfs' is not defined

## Importing Essential Libraries for Analysis and Visualization

In this section, we import the main Python libraries used for data analysis and visualization.

In [ ]:
%pip install scipy


# Fit linear and nonlinear models for temporary impact g_t(x)

from scipy.optimize import curve_fit



def linear_model(x, beta):

    return beta * x



def nonlinear_model(x, alpha, p):

    return alpha * np.power(x, p)



fit_results = {}



for ticker, df in ticker_dfs.items():

    if 'price' in df.columns and 'volume' in df.columns:

        print(f'\n--- Model fitting for {ticker} ---')

        df = df.copy()

        df['impact'] = df['price'].diff().fillna(0)

        x = df['volume'].values

        y = df['impact'].values

        # Fit linear model

        try:

            popt_lin, _ = curve_fit(linear_model, x, y)

        except Exception as e:

            print(f'Error in linear model: {e}')

            continue

        # Fit nonlinear model

        try:

            popt_nonlin, _ = curve_fit(nonlinear_model, x, y, bounds=([0,0],[np.inf,2]))

        except Exception as e:

            print(f'Error in nonlinear model: {e}')

            continue

        fit_results[ticker] = {'linear': popt_lin, 'nonlinear': popt_nonlin}

        # Plot results

        plt.figure(figsize=(7,4))

        plt.scatter(x, y, alpha=0.3, label='Data')

        x_fit = np.linspace(x.min(), x.max(), 100)

        plt.plot(x_fit, linear_model(x_fit, *popt_lin), label='Linear Model', color='royalblue')

        plt.plot(x_fit, nonlinear_model(x_fit, *popt_nonlin), label='Nonlinear Model', color='darkorange')

        plt.title(f'Temporary Impact Fitting for {ticker}')

        plt.xlabel('Volume')

        plt.ylabel('Price Change')

        plt.legend()

        plt.show()

        print(f'Linear model coefficient (beta): {popt_lin[0]:.4g}')

        print(f'Nonlinear model coefficients (alpha, p): {popt_nonlin[0]:.4g}, {popt_nonlin[1]:.4g}')

## Data Loading and Preprocessing

Here, we load the market data and perform necessary preprocessing steps to prepare it for analysis.

## Temporary Impact Modeling and Visualization

We analyze the temporary market impact and visualize the results using the fitted power-law model.

In [ ]:
# Example: Optimal Execution with Nonlinear Impact
from scipy.optimize import minimize

S = 10000  # total shares
N = 10     # intervals
alpha = 0.01
p = 0.8
P0 = 100

def cost(x):
    x = np.array(x)
    return np.sum(P0 * x + alpha * np.power(x, p))

# Constraint: sum(x) = S
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - S})
# Bounds: x_i >= 0
bounds = [(0, None)] * N
x0 = np.ones(N) * (S / N)

res = minimize(cost, x0, bounds=bounds, constraints=cons)
print('Optimal x:', res.x)
print('Total cost:', res.fun)